FACTS: <br>
Number of images in training set: N = 416 <br>
Number of Pixels per image: D = 2576 <br>
Number of classes: c = 52 <br>
-> <br>
Sw: rank(Sw)= N-c = 364 -> Mpca <= 364 <br>
Sb: rank(Sb)= c-1 = 51 -> Mlda <= 51 <br>

In [1]:
# Imports
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
# Define constants
TRAIN_RATIO = 0.8
CLASS_SIZE = np.amax(face_label) # number of classes
PIC_DIM = (46,56) # dimensions of picture
PIC_SIZE = PIC_DIM[0]*PIC_DIM[1]

NameError: name 'face_label' is not defined

In [ ]:
# Load the training and testing sets: 80% is training, 20% is testing

splittype = 'whole'

if (splittype == 'whole'):
    training = np.loadtxt('split_whole_train.gzip', dtype = 'uint8')
    test = np.loadtxt('split_whole_test.gzip', dtype = 'uint8')
elif (splittype == 'class'):
    training = np.loadtxt('split_class_train.gzip', dtype = 'uint8')
    test = np.loadtxt('split_class_test.gzip', dtype = 'uint8')

training, testing: axis 0: pictures, axis 1: pixels + label

In [ ]:
#order training data depending on their class
training = training[np.argsort(training[:,-1]),:]

# Create a list of arrays depending on the classes
class_sets = []
for i in range(52):
    class_sets.append(training[i==(training[:,-1]-1),:2576])
    
print(class_sets[0],class_sets[0].shape)

# Calculate the class means
#class_means = []
#for c in class_sets:
#    class_means.append(np.mean(c, axis=0))
#print(class_means[0],class_means[0].shape)

class_means = np.zeros((52,2576))
for i in range(52):
    class_means[i,:] = np.mean(class_sets[i], axis=0)

# Calculate the global mean
global_mean = np.mean(training[:,:2576], axis=0)
print(global_mean,global_mean.shape)

#Calculate Sb
class_means_norm = class_means - global_mean #make us of broadcasting
Sb = np.dot(class_means_norm.T, class_means_norm)
#print(np.linalg.matrix_rank(Sb)) # =c-1


In [ ]:
#Calculate x-mi
class_sets_norm = []
for i in range(52):
    class_sets_norm.append(class_sets[i]-class_means[i,:])
print(class_sets_norm[0].shape)

In [ ]:
#Calculate Sw
Sw = np.zeros((2576,2576))
for c in class_sets_norm:
    Sw += np.dot(c.T,c)
    
#print(np.linalg.matrix_rank(Sw)) # = N-c

In [ ]:
#Calculate St
# Images are represented as rows of the array 'pca_train'
pca_train = training [:, :2576]
pca_train_norm = pca_train - global_mean
St = np.dot(pca_train_norm.T, pca_train_norm)

In [ ]:
# Calculate Wpca
eigvals_St, eigvecs_St = np.linalg.eig(St)

# take real part of the eigenvalues (complex eigenvalues are the result of calculation rounding errors)
eigvals_St = np.real(eigvals_St)
eigvecs_St = np.real(eigvecs_St)

#order eigenvectors and eigenvalues according to their size
index_St = eigvals_St.argsort()[::-1]
eigvals_St = eigvals_St[index_St]
eigvecs_St = eigvecs_St[:,index_St]

# We just keep the first n eigenvectors and eigenvalues
Mpca = 350 #<=364
eigvecs_St_best = eigvecs_St[:, :Mpca]
eigvals_St_best = eigvals_St[:Mpca]

#Define Wpca
Wpca = eigvecs_St_best

print(Wpca, Wpca.shape)

In [ ]:
#calculate inverse of Sw_reduced times Sb_reduced
LDA = np.linalg.inv(Wpca.T.dot(Sw).dot(Wpca)).dot(Wpca.T.dot(Sb).dot(Wpca))

print(LDA, LDA.shape)

In [ ]:
#calculate Wlda
#Get the generalised eigenvectors of LDA_matrix with largest Mlda eigenvalues
eigvals_LDA, eigvecs_LDA = np.linalg.eig(LDA)

# take real part of the eigenvalues (complex eigenvalues are the result of calculation rounding errors)
eigvals_LDA = np.real(eigvals_LDA)
eigvecs_LDA = np.real(eigvecs_LDA)

#order eigenvectors and eigenvalues according to their size
index_LDA = eigvals_LDA.argsort()[::-1]
eigvals_LDA = eigvals_LDA[index_LDA]
eigvecs_LDA = eigvecs_LDA[:,index_LDA]

x = np.arange(0, 350)
print (eigvals_LDA, eigvals_LDA.shape)

# plot them
plt.subplot(3, 1, 1)
plt.plot(x, eigvals_LDA) # clearly out of scale, so let's just take the first 200 of them
plt.title('All eigenvalues of the covariance matrix')

plt.subplot(3, 1, 2)
plt.plot(x[:200], eigvals_LDA[:200]) # still out of scale, so let's just take the first 20 of them
plt.title('First 200 eigenvalues of the covariance matrix')

plt.subplot(3, 1, 3)
plt.plot(x[:20], eigvals_LDA[:20])
plt.title('First 20 eigenvalues of the covariance matrix')
plt.show()

# We just keep the first Mlda eigenvectors and eigenvalues
Mlda = 50 #<=51
eigvecs_LDA_best = eigvecs_LDA[:, :Mlda]
eigvals_LDA_best = eigvals_LDA[:Mlda]

#Define Wpca
Wlda = eigvecs_LDA_best

print(Wlda, Wlda.shape)

In [ ]:
#Calculate Wopt
Wopt_transposed = Wlda.T.dot(Wpca.T)
print(Wopt_transposed,Wopt_transposed.shape)
Wopt = Wopt_transposed.T
plt.imshow(np.reshape(Wopt[:,0], (46,56)).T, cmap = 'gist_gray')

In [ ]:
#Reconstruction check
plt.imshow(np.reshape(pca_train[0,:], (46,56)).T, cmap = 'gist_gray')
pic_LDA = Wopt_transposed.dot((pca_train[0,:]-global_mean))
pic_LDA_reconstructed = Wopt.dot(pic_LDA)+global_mean
plt.figure()
plt.imshow(np.reshape(pic_LDA_reconstructed, (46,56)).T, cmap = 'gist_gray')

In [ ]:
#Predict the class for the test set
#Project train data into LDA space
train_norm = (training[:,:2576] - global_mean).T 
print(train_norm.shape) #each picture is a column
train_LDA = Wopt_transposed.dot(train_norm)
print(train_LDA.shape) #each picture is a column

#Project test data into LDA space
test_norm = (test[:,:2576] - global_mean).T 
print(test_norm.shape) #each picture is a column
test_LDA = Wopt_transposed.dot(test_norm)
print(test_LDA.shape) #each picture is a column

train_LDA_reshape_1 = np.repeat(train_LDA.reshape(Mlda,416,1),104, axis=2) #axis-0: projected values, axis-1: training pictures, axis-2: repeated values
test_LDA_reshape_1 = np.repeat(test_LDA.reshape(Mlda,1,104), 416, axis=1) #axis-0: projected values, axis-1: repeated values, axis-2: test pictures

#Do nearest neighbours
distances = np.linalg.norm(test_LDA_reshape_1-train_LDA_reshape_1, axis=0)
print(distances.shape)

#calculate the nearest training picture
nearest_trainpicture = distances.argmin(axis=0)
print(nearest_trainpicture)

#determine the nearest classes
classes_pred = training[nearest_trainpicture,2576]
classes_act = test[:,2576]

cm = confusion_matrix(classes_act, classes_pred)

print(cm)
plt.matshow(cm, cmap = 'Blues')
plt.colorbar()
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

percentage = np.sum(classes_pred==classes_act) / len(classes_act)
print("Percentage: ",percentage)


In [ ]:
#Predict the class for the test set (average nearest neighbours)
#Project all classes into the LDA-Space
class_sets_LDA = []
for c in class_sets:
    class_sets_LDA.append(Wopt_transposed.dot((c-global_mean).T))
print(class_sets_LDA[0], class_sets_LDA[0].shape)

#calculate the average distance to the class
class_mean_distance = np.zeros((52,104))
for i in range(52):
    train_LDA_reshape_2 = np.repeat(class_sets_LDA[i].reshape(Mlda,class_sets_LDA[i].shape[1],1),104,axis=2)
    test_LDA_reshape_2 = np.repeat(test_LDA.reshape(Mlda,1,104), class_sets_LDA[i].shape[1], axis=1)
    d = np.linalg.norm(test_LDA_reshape_2-train_LDA_reshape_2, axis=0)
    class_mean_distance[i,:]=np.average(d,axis=0)
    
#class_mean_distance is a matrix where each element stores the mean class difference of a test point to a certain class
#axis-0: classes, axis-1: mean distance for each point
print(class_mean_distance.shape)



In [ ]:
#determine the nearest classes
classes_pred_1 = class_mean_distance.argmin(axis=0)+1 #python index starts from 0 -> add 1
classes_act_1 = test[:,2576]

cm = confusion_matrix(classes_act_1, classes_pred_1)

print(cm)
plt.matshow(cm, cmap = 'Blues')
plt.colorbar()
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

percentage_1 = np.sum(classes_pred_1==classes_act_1) / len(classes_act_1)
print("Percentage: ",percentage_1)